In [1]:
#sample sentences. should be parseable
s1 = "humid in the morning."
s2 = "windy and mostly cloudy tonight."
s3 = "heavy drizzle (4-6 in.) throughout the day."

In [2]:
import bigsky.cky as cky
import bigsky.forecast as fc
import bigsky.cfg as cfg

In [3]:
g = cfg.Cfg.from_file("data/cfgs/weather.cfg")
g = g.binarize()

In [4]:
cky.cky_parse(s1, g)

True

In [5]:
t1 = cky.cky_tree(s1, g)
print(t1)
t2 = cky.cky_tree(s2, g)
print(t2)
t3 = cky.cky_tree(s3, g)
print(t3)

[('humid', [([('in', [('the', 'morning')])], '.')])]
[([('windy', [('and', [('mostly', 'cloudy')])])], [('tonight', '.')])]
False


In [12]:
import json
'''
put the resulting string into this syntax tree viewer:
http://ironcreek.net/syntaxtree/
(Turning off colors and autosubscripts is advisable)
'''
def reformat_tree(t):
    s = json.dumps(t)
    s = s.replace('"', '')
    s = s.replace(',','')
    s = s.replace('[[','[')
    s = s.replace(']]',']')
    s = s.replace('[','[+ ')
    print(s)

reformat_tree(t1)
reformat_tree(t2)

[+ humid [+ [+ in [+ the morning]] .]]
[+ [+ windy [+ and [+ mostly cloudy]]] [+ tonight .]]
